In [ ]:
import boto3
import pickle
from tqdm.notebook import tqdm
import json
from collections import defaultdict
import time

Setting up boto3 credentials: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html

In [ ]:
# Need to set up AWS credentials or pass them here as arguments 

client = boto3.client('transcribe')

In [ ]:
response = client.get_transcription_job(
    TranscriptionJobName='Transcription-test-yshao-8.wav'
)
response

In [ ]:
response['TranscriptionJob']['Transcript']

In [ ]:
response = client.start_transcription_job(
    TranscriptionJobName='Transcription-test-yshao-8.wav',
    LanguageCode='en-US',
#     MediaSampleRateHertz=16000,
    MediaFormat='wav',
    Media={
        'MediaFileUri': 's3://spider-questions-audio/human-test/yshao-wav/8.wav'
    },
    OutputBucketName='spider-audio-transcription',
#     OutputEncryptionKMSKeyId='string',
    Settings={
#         'VocabularyName': 'string',
#         'ShowSpeakerLabels': True|False,
#         'MaxSpeakerLabels': 123,
#         'ChannelIdentification': True|False,
        'ShowAlternatives': True,
        'MaxAlternatives': 10,
#         'VocabularyFilterName': 'string',
#         'VocabularyFilterMethod': 'remove'|'mask'
    },
)
response

In [ ]:
# Do transcribe 
response_dict = defaultdict(dict)

In [ ]:
for fileID in tqdm(range(100)):
    while True:
        # Wait 2 secs for completion
        time.sleep(2)
        
        try:
            response = client.start_transcription_job(
                TranscriptionJobName='Transcription-human-test-yshao-{}.wav'.format(fileID),
                LanguageCode='en-US',
                MediaFormat='wav',
                Media={
                    'MediaFileUri': 's3://spider-questions-audio/human-test/yshao-wav/{}.wav'.format(fileID)
                },
                OutputBucketName='spider-audio-transcription',
                Settings={
                    'ShowAlternatives': True,
                    'MaxAlternatives': 10
                },
            )
            response_dict[fileID] = response
#         except LimitExceededException as e:
#             print(e)
        except Exception as e:
            raise(e)
        else:
            break    
    
len(response_dict)

In [ ]:
len(response_dict)

In [ ]:
trans_json = json.load(open('spider-dev-batch0/Transcription-dev-batch0-1.wav.json', 'r'))
trans_json.keys()

In [ ]:
trans_json['results'].keys()

In [ ]:
[a['transcript'] for a in trans_json['results']['segments'][0]['alternatives']]